# PyTorch로 시작하는 딥러닝

## 소프트맥스

### 비용함수 구현

In [6]:
import torch
import torch.nn.functional as F

In [7]:
torch.manual_seed(1)

In [8]:
z = torch.FloatTensor([1,2,3])
print(z)

tensor([1., 2., 3.])


In [9]:
hypothesis = F.softmax(z, dim=0)
print(hypothesis)
print(hypothesis.sum())

tensor([0.0900, 0.2447, 0.6652])
tensor(1.)


In [10]:
# random하게 (3,5) size tensor를 정의 
z = torch.rand(3, 5, requires_grad=True)
print(z)

tensor([[0.7576, 0.2793, 0.4031, 0.7347, 0.0293],
        [0.7999, 0.3971, 0.7544, 0.5695, 0.4388],
        [0.6387, 0.5247, 0.6826, 0.3051, 0.4635]], requires_grad=True)


In [11]:
hypothesis = F.softmax(z, dim = 1)
print(hypothesis)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward>)


In [12]:
hypothesis = F.softmax(z, dim = 0)
print(hypothesis)

tensor([[0.3412, 0.2938, 0.2671, 0.4002, 0.2469],
        [0.3559, 0.3306, 0.3796, 0.3393, 0.3719],
        [0.3029, 0.3756, 0.3533, 0.2605, 0.3812]], grad_fn=<SoftmaxBackward>)


대체로 함수에서 dim을 적는 경우가 있다. 이 경우 어떻게 해야 헷갈리지 않을까를 고민해봤다.  
그 차원으로 계산을 해주세요라고 생각하면 헷갈리기 시작하므로 그냥 그 챠원은 고정하고 나머지 차원을 계산한다고 생각한다.  

ex. 2차원 tensor를 가정. dim = 0이면 행은 고정하고 열에다가 연산 적용, dim = 1이면 열은 고정하고 행에다가 연산 적용.

In [13]:
y = torch.randint(low=0, high=5, size=(3,)).long()
print(y)

tensor([0, 2, 1])


In [15]:
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1),1)
# scatter 함수가 어려울 수 있음 아래의 사이트 보면서 항상 까먹지 말자
# https://aigong.tistory.com/35

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [17]:
cost = (y_one_hot*-torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.0078, grad_fn=<MeanBackward0>)


In [18]:
a = torch.log(F.softmax(z, dim=1))
b = F.log_softmax(z, dim=1)
print(a)
print(b)
# 결과는 동일

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward>)
tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward>)


In [21]:
cost1 = (y_one_hot*-torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()
cost2 = (y_one_hot*-F.log_softmax(z, dim=1)).sum(dim=1).mean()
cost3 = F.nll_loss(F.log_softmax(z, dim=1), y)
cost4 = F.cross_entropy(z,y)

print(cost1)
print(cost2)
print(cost3)
print(cost4)

tensor(1.4689, grad_fn=<MeanBackward0>)
tensor(1.4689, grad_fn=<MeanBackward0>)
tensor(1.4689, grad_fn=<NllLossBackward>)
tensor(1.4689, grad_fn=<NllLossBackward>)


### 소프트맥스 회귀 구현 일반

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

torch.manual_seed(1)

x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

y_one_hot = torch.zeros(8,3)
y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)

W = torch.zeros((4,3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W,b], lr = 0.1)
total_epoch = 1000

for epoch in range(total_epoch+1):

    hypothesis = F.softmax(x_train.matmul(W)+b, dim = 1)

    cost = (y_one_hot*-torch.log(hypothesis)).sum(dim=1).mean()

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, total_epoch, cost.item()))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

torch.manual_seed(1)

x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

y_one_hot = torch.zeros(8,3)
y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)

W = torch.zeros((4,3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
optimizer = optim.SGD([W,b], lr = 0.1)

total_epoch = 1000
for epoch in range(total_epoch + 1):

    z = x_train.matmul(W)+b
    cost = F.cross_entropy(z, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0 :
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, total_epoch, cost.item()))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962
